# Azure AI Agent Service மற்றும் Semantic Kernel பயன்படுத்தி RAG

இந்த குறியீட்டு துணுக்கில் `Azure AI Agent Service` மற்றும் `Semantic Kernel` பயன்படுத்தி retrieval-augmented generation (RAG) செய்ய Azure AI முகவரியை உருவாக்குவது மற்றும் நிர்வகிப்பது எப்படி என்பதை விளக்குகிறது. இந்த முகவர் பயனர் கேள்விகளை பெறப்பட்ட சூழலின் அடிப்படையில் செயல்படுத்தி துல்லியமான பதில்களை வழங்குகிறது.


## சூழலை ஆரம்பித்தல்


SQLite பதிப்பு சரிசெய்தல்  
நீங்கள் பின்வரும் பிழையை சந்தித்தால்:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```
  
உங்கள் நோட்புக் தொடக்கத்தில் இந்த குறியீட்டு தொகுதியை Uncomment செய்யவும்:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### பாக்கேஜ்களை இறக்குமதி செய்வது  
கீழே உள்ள குறியீடு தேவையான பாக்கேஜ்களை இறக்குமதி செய்கிறது:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# செமாண்டிக் கர்னல் மற்றும் Azure AI Agent Service உடன் Retrieval-Augmented Generation

இந்த மாதிரி **Azure AI Agent Service**-ஐ பயன்படுத்தி **Retrieval-Augmented Generation (RAG)** செய்யும் முறையை, மொழி மாதிரியை பதிவேற்றப்பட்ட ஆவணத்தின் துறையால் குறிப்பிட்ட உள்ளடக்கத்துடன் இணைத்து, விளக்குகிறது.

### இது எப்படி செயல்படுகிறது

1. **ஆவண பதிவேற்றம்**: ஒரு markdown கோப்பு (document.md) தகவல்களுடன் (Contoso-வின் பயண காப்பீட்டு கொள்கை) agent சேவைக்கு பதிவேற்றப்படுகிறது.

2. **வெக்டர் சேமிப்பு உருவாக்கம்**: ஆவணம் அதன் உள்ளடக்கத்தில் செமாண்டிக் தேடலை செயல்படுத்த, வெக்டர் சேமிப்பில் குறியிடப்படுகிறது.

3. **Agent அமைப்பு**: `gpt-4o` மாதிரியை பயன்படுத்தி ஒரு agent உருவாக்கப்படுகிறது, இதற்கான கடுமையான வழிமுறைகள்:
   - ஆவணத்தில் இருந்து பெறப்பட்ட உள்ளடக்கத்தின் அடிப்படையில் மட்டுமே கேள்விகளுக்கு பதிலளிக்க வேண்டும்.
   - கேள்வி வரம்புக்கு வெளியே இருந்தால் பதிலளிக்க மறுக்க வேண்டும்.

4. **கோப்பு தேடல் கருவி ஒருங்கிணைப்பு**: `FileSearchTool` agent-இன் மூலம் பதிவு செய்யப்படுகிறது, இது மாதிரிக்கு குறியிடப்பட்ட ஆவணத்தில் இருந்து தொடர்புடைய பகுதிகளை தேடவும் பெறவும் உதவுகிறது.

5. **பயனர் தொடர்பு**: பயனர்கள் கேள்விகளை கேட்கலாம். ஆவணத்தில் தொடர்புடைய தகவல் கிடைத்தால், agent ஒரு உறுதிப்படுத்தப்பட்ட பதிலை உருவாக்குகிறது.  
   தகவல் போதுமானதாக இல்லையெனில், ஆவணத்தில் போதுமான தகவல் இல்லை என்று agent தெளிவாக பதிலளிக்கிறது.


### முக்கிய செயல்பாடு


முதலில் Azure CLI-யை பயன்படுத்தி `az login` ஐ இயக்கவும், இதனால் `DefaultAzureCredential` பயன்படுத்தும் போது சரியான அங்கீகார சூழல் வழங்கப்படும். Azure AI Agent Service API விசைகளை பயன்படுத்தாது.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

நீங்கள் இதற்குச் சமமான வெளியீட்டை காணலாம்:

```
# User: 'Can you explain Contoso's travel insurance coverage?'
# Agent: Contoso's travel insurance coverage includes protection for medical emergencies, trip cancellations, and lost baggage【4:0†document.md】.
# User: 'What is a Neural Network?'
# Agent: I'm sorry, but the uploaded document does not contain the necessary information to answer that question.

Cleaned up agent, thread, file, and vector store.
```



---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கிறோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை கவனத்தில் கொள்ளவும். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
